# Conten Based filtering using audio features, album_uri and artist_uri

In [1]:
import import_ipynb

In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import scipy.sparse as sps

from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from category_encoders import TargetEncoder
from time import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#own functions
from evaluation import DCG
from evaluation import nDCG
from evaluation import R_Precision


%matplotlib inline

importing Jupyter notebook from evaluation.ipynb
DCG = 0.5
IDCG = 1.0
nDCG = 0.5


# Define function to look up similar items 

In [3]:
with open('../data-processed/full-data/track_descriptions.json') as json_file:
    D_desc = json.load(json_file)
    
D_desc['spotify:track:0UaMYEvWZi0ZqiDOoHU3YI']

['Lose Control (feat. Ciara & Fat Man Scoop)', 'Missy Elliott', 'The Cookbook']

In [4]:
def similar_items(track_uri, top_n, similarity_matrix):
    #print('seed track is not included')
    D_rec={}
    #time0 = time()
    idx = D_track_uri_to_id[track_uri]
    similarity_array = cosine_similarity(similarity_matrix[idx:idx+1,:], similarity_matrix)
    recc_tracks = np.fliplr(similarity_array.argsort())
    for el in recc_tracks[0][1:top_n+1]:
        D_rec[D_track_id_to_uri[el]]=similarity_array[0][el]
    #print(f'{time()-time0:0.2f}sec')
    return D_rec

In [5]:
def similar_items_with_description(track_uri, top_n, similarity_matrix):
    print('CB audio features - first tracks returned is the seed track')
    D_rec={}
    time0 = time()
    idx = D_track_uri_to_id[track_uri]
    similarity_array = cosine_similarity(similarity_matrix[idx:idx+1,:], similarity_matrix)
    recc_tracks = np.fliplr(similarity_array.argsort())
    for el in recc_tracks[0][0:top_n+1]:
        D_rec[D_track_id_to_uri[el]]=D_desc[D_track_id_to_uri[el]]
    print(f'{time()-time0:0.2f}sec')
    return list(D_rec.values())

In [6]:
def similar_items_with_description_external(track_uri, top_n):
    print('CB audio features - first tracks returned is the seed track')
    D_rec={}
    time0 = time()
    idx = D_track_uri_to_id[track_uri]
    similarity_array = cosine_similarity(X_transformed[idx:idx+1,:], X_transformed)
    recc_tracks = np.fliplr(similarity_array.argsort())
    for el in recc_tracks[0][0:top_n+1]:
        D_rec[D_track_id_to_uri[el]]=D_desc[D_track_id_to_uri[el]]
    print(f'{time()-time0:0.2f}sec')
    return list(D_rec.values())

In [7]:
import operator

def predict_and_evaluate_top_n(pid,top_n):
    """
    return
    (1) top_n predicted track_ids
    (2) ground_truth : track_ids in the hold_out
    (3) R_Prec
    
    """
    train_array_track_ids = track_id_array[M[pid].toarray()[0].astype(bool)]
    
    D_pred={}
    
    topn_n_by_track = int(top_n/len(train_array_track_ids))*2
    
    for el in train_array_track_ids:
        D_pred.update(similar_items(D_track_id_to_uri[el],topn_n_by_track,X_transformed))
    
    D_pred = dict(sorted(D_pred.items(), key=operator.itemgetter(1),reverse=True))
    ground_truth = ev_set_arr[ev_set_arr[:,0]==pid][:,2]
    L_pred = list(D_pred.keys())[:top_n]
    R_Prec = R_Precision(L_pred[:len(ground_truth)],ground_truth)
    res = [int(el in list(ground_truth)) for el in L_pred]
    NDCG = nDCG(res)[1]
    return L_pred, ground_truth, R_Prec, NDCG, res

# Data transformation, PCA and merging 

In [8]:
df_merged = pd.read_csv('../data-processed/transformation-matrices/cb_df_merged.csv')

In [9]:
df_merged.shape

(2262190, 836)

In [10]:
df_merged.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,pca_genre_791,pca_genre_792,pca_genre_793,pca_genre_794,pca_genre_795,pca_genre_796,pca_genre_797,pca_genre_798,pca_genre_799,pca_genre_artist_uri
0,0.897250,-0.616004,0.488537,-0.337567,-1.377756,-0.535840,0.830192,-0.626090,-0.608544,-0.995276,...,0.014805,0.020956,-0.012763,0.016099,-0.013033,0.071684,0.053541,-0.013224,-0.023828,spotify:artist:1vSHzGHsVOCrgPSCmKNimP
1,-2.218729,-0.540820,-0.073421,-0.574500,0.725818,-0.000277,0.974099,1.334160,-0.427320,-1.493465,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,1.347035,1.158320,0.769516,0.740272,0.725818,-0.223356,-1.001042,1.983828,0.805424,0.197414,...,-0.000283,-0.008057,0.009964,0.006024,-0.013305,0.009614,0.005862,0.033953,0.007343,spotify:artist:7kxOVclB0zQamtBR0syCrg
3,1.292844,-0.138590,-0.354400,0.443262,0.725818,-0.411715,0.993851,-0.629341,-0.777124,0.434471,...,-0.008368,-0.000212,0.002130,-0.014916,-0.015205,-0.000168,0.002519,-0.008507,-0.013943,spotify:artist:2sxmKe3CUrWnx7eoXMhOlW
4,0.962279,0.320027,-0.635379,0.310843,-1.377756,0.110828,-0.558085,-0.629734,-0.532682,-1.522356,...,-0.000859,0.001006,-0.002154,0.003383,0.000224,-0.001266,0.001580,-0.003220,0.003751,spotify:artist:6PyeXqjH8OMGnt1IOhWgrQ


In [ ]:
#df_merged = df_merged.fillna(0)

In [ ]:
#df_merged.to_csv('../data-processed/transformation-matrices/cb_df_merged.csv', index = None)

In [1]:
X_transformed = np.asarray(df_merged)

In [ ]:
X_transformed.shape

# Find similar tracks

<img src="tenor.gif">

In [ ]:
similar_items_with_description('spotify:track:3ZOEytgrvLwQaqXreDs2Jx',10,X_transformed)